# Scattering wavefunctions in momentum space

We consider here again the momentum-space Lippmann-Schwinger equation in a fixed partial wave,

\begin{equation}
T(E_k+\mathrm{i}\varepsilon;p,k) = V(p,k)
+\int\frac{\mathrm{d}q\,q^2}{2\pi^2} V(p,q)
G_0(E_k+\mathrm{i}\varepsilon;q)
T(E_k+\mathrm{i}\varepsilon;q,p) \,,
\end{equation}

with $\varepsilon\to0$ implied, and calculate **scattering wavefunctions** from the solutions of this equation according to

\begin{equation}
 \psi^{(+)}_k(q)
 = \frac{2\pi^2\delta(q-k)}{k^2} + \frac{2\mu T(E_k;q,k)}{k^2 - q^2 + \mathrm{i}\varepsilon} \,.
\end{equation}

## System and potential

We consider here again our previously introduced simple system (mass $m=1$ and $\hbar=1$), and Gaussian potential:

In [ ]:
from lib.system import *
from lib.potential import *

sys = System()
V = V_Gauss(sys, -4.0, 2.0)

## Equation solving

We copy the basic infrastructure for solving the half off-shell Lippmann-Schwinger equation from what we used before:

In [ ]:
from lib.mesh import *
from lib.operator import *

mesh = GaulegMesh(16, 0.0, 4.0)

G0 = G_0(sys)

In [ ]:
def kernel(E):
  factor = 0.5 / np.pi**2

  K = map(
    lambda p: list(map(
      lambda qw: factor * qw[1] * qw[0]**2 \
        * (G0.residue(qw[0]) if qw[2] else G0(E, qw[0])) \
        * V.get(0, p, qw[0]),
      mesh.pws()
    )), mesh.ps()
  )

  return np.asarray(list(K))

In [ ]:
def solve(k):
  mesh.push_pv(k)
  
  mat = np.identity(mesh.size()) - kernel(sys.e_from_k(k))
  vec = np.asarray(list(map(lambda p: V.get(0, p, k), mesh.ps())))
  
  sol = np.linalg.solve(mat, vec)
  
  mesh.pop_pv()
  
  return sol;

`solve` then gives us $T(E_k;p,k)$ for fixed $k$ and disretized $p$.

## Fourier transformation

We can calculate the more familiar configuration space wavefunction $\psi^{(+)}_k(r)$ as the **Fourier-Bessel transform** of $\psi^{(+)}_k(q)$:

\begin{equation}
 \psi^{(+)}_k(r) = \frac{1}{2\pi^2}\int \mathrm{d}q\,q^2 j_0(qr) \psi^{(+)}_k(q)
\end{equation}

Since we are considering an S-wave state, the integral involves the zero-th order spherical Bessel function $j_0$.

To compare with a solution of the radial Schrödinger equation for the same potential, what we actually want is the **reduced radial wavefunction** $u(r) = r\,\psi^{(+)}_k(r)$.  Note that since we are working in partial waves, $\psi^{(+)}$ here already has the angular dependence factored out -- for the S-wave case this means there is no additional factor $Y_0^0 = 1/\sqrt{4\pi}$.

We can calculate $u(r)$ directly by working with a Riccati-Bessel function:

\begin{equation}
 u(r) = \frac{1}{2\pi^2}\int \mathrm{d}q\,q \,\hat{j_0}(qr) \psi^{(+)}_k(q)
\end{equation}

Note that there is now also only a single power of $q$ left because $j_0(qr) = \hat{j_0}(qr)/(qr)$.

Overall, we have the following expression:

\begin{equation}
 u(r) = \frac{1}{2\pi^2}\int \mathrm{d}q\,q \,\hat{j_0}(qr)
 \left[\frac{2\pi^2\delta(q-k)}{k^2} + \frac{2\mu T(E_k;q,k)}{k^2 - q^2 + \mathrm{i}\varepsilon}\right]
\end{equation}

The first part of this we can directly evaluate to find

\begin{equation}
 u(r) = \hat{j_0}(kr)/k + \cdots \,,
\end{equation}

with the $\cdots$ left to be evaluated numerically.

Clearly that evaluation again involves a principal-value integration and in order to use our existing infrastructure for that it makes sense to write it again in terms of the Green's function:

\begin{equation}
 \frac{2\mu T(E_k;q,k)}{k^2 - q^2 + \mathrm{i}\varepsilon} = G_0(E_k+\mathrm{i}\varepsilon;q) T(E_k;q,k)
\end{equation}

We finally arrive at the following implementation:

In [ ]:
from lib.riccati import *

def u(sol, k, r):
  mesh.push_pv(k)
  
  acc = 0
  for qw, t in zip(mesh.pws(), sol):
    acc += qw[1] * qw[0] \
      * riccati_j(0, qw[0] * r) \
      * (G0.residue(qw[0]) if qw[2] else G0(sys.e_from_k(k), qw[0])) \
      * t
    
  mesh.pop_pv()
    
  return riccati_j(0, k * r) / k + 0.5 * acc / np.pi**2

In [ ]:
mesh = GaulegMesh(64, 0.0, 4.0)

k = 1.0

sol_k = solve(k)

rs = np.linspace(0.0001, 20.0, 1000)
us = list(map(lambda r: u(sol_k, k, r), rs))

plt.plot(rs, np.real(us))
plt.show()

## Comparison

Let us compare this result to the direct calculation in configuration space:

In [ ]:
from scipy.integrate import odeint

def radseq(uv, r, k, l):
  u, v = uv
  return [v, (l * (l + 1) / r**2 + 2.0 * sys.mu * V(r) - k**2) * u]

uv0 = [0.0, 1.0]

sol_r = odeint(radseq, uv0, rs, args=(k, 0))

plt.plot(rs, sol_r[:, 0])
plt.plot(rs, np.real(us))
plt.show()

We can see that the two wavefunctions differ in their normalization, but otherwise they are in good agreement, an in particular their nodes coincide.  Indeed, picking a particular node $r_0$, we can use

\begin{equation}
 \tan\delta(p) = {-}\frac{\hat{j}_0(pr_0)}{\hat{n}_0(pr_0)}
\end{equation}

to calculate the phase shift from either solution and find good agreement:

In [ ]:
from scipy.interpolate import interp1d
from scipy.optimize import root_scalar

def delta(us):
  u = interp1d(rs, us)

  root = root_scalar(u, method='secant', x0=10.0, x1=10.1)
  r0 = root.root

  print(root)

  return np.arctan(-riccati_j(0, k * r0) / riccati_n(0, k * r0))

print(delta(sol_r[:, 0]))
print(delta(np.imag(us)))

Finally, let us analyze in what way exactly the normalizations are different.

For our configuration-space solution, the initial condition `uv0 = [0.0, 1.0]` normalizes the solution such that it has unit slope at the origin.  Naively, from the first term $\hat{j_0}(kr)/k$ we might expect the same for the Fourier-transformed momentum-space solution, but the second term also contributes to all $r$, of course.

In fact, we find that our momentum-space calculation gives a **complex** wavefunction:

In [ ]:
plt.plot(rs, np.real(us))
plt.plot(rs, np.imag(us))
plt.show()

The phase turns out to be exaclty the scattering phase shift!  We can rotate it away and obtain a real wavefunction:

In [ ]:
d = delta(np.real(us))

us0 = list(map(lambda u: np.exp(-1j * d) * u, us))

plt.plot(rs, np.real(us0))
plt.plot(rs, np.imag(us0))
plt.show()

Moreover, we see that the result now has unit amplitude asymptotically, so we conclude that our $u(r)$ obtained from transforming the momentum-result is normalized such that

\begin{equation}
 u(r) = \mathrm{e}^{\mathrm{i}\delta_0(k)}\sin(kr + \delta(k))
\end{equation}

for $r\to\infty$.  In other words, it is is precisely what we call the **normalized solution**, denoted by
$u_{l,k}(r)$ for a general partial wave $l$.

What we calculate with initial condition `uv0 = [0.0, 1.0]` in configuration space is, in turn, the **regular solution as defined by Newton**, $\varphi(r)$, related to **Taylor's regular solution** via $\varphi(r) = \phi_{l,k}(r) / k$.

Overall we conclude that the normalization difference between the wavefunctions obtained from the 
two approaches is exactly the **Jost function** $J_0^+(k)$, since we know that

\begin{equation}
 u_{l,k}(r) = \frac{\phi_{l,k}(r)}{J_0^+(k)} \,.
\end{equation}